## 這篇介紹tf.Print() 的用法

## ex1）

1）查看電腦裡的tensorflow 支援的是新版的tf.print() 或是tf.Print()

In [2]:
import tensorflow as tf

In [3]:
tf.Print.__name__

'Print'

In [4]:
tf.print.__name__

AttributeError: module 'tensorflow' has no attribute 'print'

##### <ul><li>小寫的tf.print. _name__ 傳回 AttributeError。</li> <li>代表我的tensorflow 裡沒有小寫的tf.print。我的tensorflow 用的是大寫的tf.Print()。</li><li>如果電腦裡的是tf_print，則需要將 "x1=tf.Print(x,[x])",  拿掉 “x1=”,  只剩 "tf.print(x,[x])"，這樣就好。</li><li>不過這樣就不能sess.run(x1)了。不知道要怎麼改</li></ul>

2）開始

In [5]:
def graph(input):
    x=input[0]
    x1= tf.Print(x,[x],message="tf pirnt node",name="julien_node")
    print('x= ',x)
    print('x1= ',x1)
    print('x1.name = ',x1.name)
    return x1
    

In [6]:
with tf.Session() as sess:
    input = tf.constant([1,2,3,4,5])
    sess.run(graph(input))
    
# equivalent to sess.run(x1)

x=  Tensor("strided_slice:0", shape=(), dtype=int32)
x1=  Tensor("julien_node:0", shape=(), dtype=int32)
x1.name =  julien_node:0


1

### 寫成py 檔
#### <ul><li>因為tf.Print()是print 到系統的stderr。所以我們要在command line 下去執行 : $ python tf_print_1.py </li><ul>

In [7]:
%%writefile tf_print_1.py
import tensorflow as tf

def graph(input):
    x=input[0]
    x1= tf.Print(x,[x],message="tf_Print is : ",name="julien_node")
    print('x= ',x)
    print('x1= ',x1)
    print('x1.name = ',x1.name)
    return x1
    

with tf.Session() as sess:
    input = tf.constant([1,2,3,4,5])
    sess.run(graph(input))
    
# equivalent to sess.run(x1)

Overwriting tf_print_1.py


### 用placeholder 寫

### <ul><li>make_graph()必須return placeholder 的namespace。</li><li>input = tf.placeholder</li><li>因為feed_dict的key 必須是global variable。</li><li>feed_dict={input: np_array}</li><li>如果make_graph()不return input 出來。input variable 是make_graph的local variable。 </li></ul>

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
def make_graph():
    input =tf.placeholder(dtype=tf.int32, shape=None)
    x=input[1]
    x1= tf.Print(x,[x],message="tf pirnt node",name="julien_print_node")
    return x1,x,input

In [10]:
with tf.Session() as sess:
    input_array = np.array([1,2,3,4,5])
    x1,_,input = make_graph()
    sess.run(x1, feed_dict={input: input_array})
    

2

## ex2）用loop 寫一次
### making 5 "dangling“ node：x1 in the same graph 。

In [11]:
def graph(input):
    node = []
    # 我想表達的是for i in range(len(input))，只不過因為input 是Tensor object, 所以寫起來就很麻煩。
    for i in range(tf.shape(input)[0].eval()):
        x=input[i]
        x1= tf.Print(x,[x],message="tf_Print is: ",name="julien_node")
        print('x= ',x)
        print('x1= ',x1)
        node.append(x1)
    return node
    

In [12]:
node = graph(tf.constant([1,2,3,4,5]))
node

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [13]:
with tf.Session() as sess:
    input = tf.constant([1,2,3,4,5])
    sess.run(graph(input))

# equivalent to sess.run([julien_node, juilen_node_2,juien_node_3,...])

x=  Tensor("strided_slice_4:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_1:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_5:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_2:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_6:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_3:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_7:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_4:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_8:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_5:0", shape=(), dtype=int32)


[1, 2, 3, 4, 5]

### ex2-1)寫成py 檔
#### <ul><li>因為tf.Print()是print 到系統的stderr。所以我們要在command line 下去執行 : $ python tf_print_2.py </li><ul>

In [14]:
%%writefile tf_print_2.py
import tensorflow as tf 

def graph(input):
    node = []
    for i in range(tf.shape(input)[0].eval()):
        x=input[i]
        x1= tf.Print(x,[x],message="tf_Print is: ",name="julien_node")
        print('x= ',x)
        print('x1= ',x1)
        node.append(x1)
    return node
    

with tf.Session() as sess:
    input = tf.constant([1,2,3,4,5])
    sess.run(graph(input))

# equivalent to sess.run([julien_node, juilen_node_2,juien_node_3,...])

Overwriting tf_print_2.py


#### <ul><li>Reviw:</li> <li>tf.Print 並沒有照list 的順序印出[1],[2],[3],[4],[5] </li> <li>我想這可能和threading的運算有關係 </li></ul>

## ex3) 用place holder 和feed_dict 再寫一次
### <ol><li>function take away parameter: input</li><li>add placeholder Tensor</li></ol>

In [15]:
import tensorflow as tf
import numpy as np

# function take away parameter: input
def make_graph():
    node = []
    
    # add place holder 
    input=tf.placeholder(dtype="int32", shape=[5])
    
    # 我想表達的是for i in range(len(input))，只不過因為input 是Tensor object, 所以寫起來就很麻煩。
    for i in range(tf.shape(input)[0].eval(session=sess)):

        x=input[i]
        x1= tf.Print(x,[x],message="tf_Print is: ",name="julien_node")
        print('x= ',x)
        print('x1= ',x1)
        node.append(x1)
    return node,input    

In [16]:
with tf.Session() as sess:
    input_array = np.array([1,2,3,4,5])
    node, placeholder = make_graph()
    sess.run(node,feed_dict={placeholder:input_array})

# equivalent to sess.run([julien_node, juilen_node_2,juien_node_3,...])

x=  Tensor("strided_slice_10:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_6:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_11:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_7:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_12:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_8:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_13:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_9:0", shape=(), dtype=int32)
x=  Tensor("strided_slice_14:0", shape=(), dtype=int32)
x1=  Tensor("julien_node_10:0", shape=(), dtype=int32)


[1, 2, 3, 4, 5]

### 寫成.py file， 去command line 執行 <ul><li>$ python tf_print_2_feeddict.py</li></ul>

In [17]:
%%writefile tf_print_2_feeddict.py

import tensorflow as tf
import numpy as np

# function take away parameter: input
def make_graph():
    node = []
    
    # add place holder 
    input=tf.placeholder(dtype="int32", shape=[5])
    
    # 我想表達的是for i in range(len(input))，只不過因為input 是Tensor object, 所以寫起來就很麻煩。
    for i in range(tf.shape(input)[0].eval(session=sess)):

        x=input[i]
        x1= tf.Print(x,[x],message="tf_Print is: ",name="julien_node")
        print('x= ',x)
        print('x1= ',x1)
        node.append(x1)
    return node,input    

with tf.Session() as sess:
    input_array = np.array([1,2,3,4,5])
    node, placeholder = make_graph()
    sess.run(node,feed_dict={placeholder:input_array})

# equivalent to sess.run([julien_node, juilen_node_2,juien_node_3,...])

Overwriting tf_print_2_feeddict.py


<h2><ul>
    <li>tf.shape(inpuit) returns shape tensor</li>
    <li>input.get_shape()returns a [tensor with value] without having to run sess </li>
    </ul></h2>

In [19]:
import tensorflow as tf

In [21]:
a = tf.constant([1,2,3,4,5])
b = tf.shape(a)
c =a.get_shape()

b
c


<tf.Tensor 'Shape_4:0' shape=(1,) dtype=int32>

TensorShape([Dimension(5)])

In [22]:
type(b)
type(c)

tensorflow.python.framework.ops.Tensor

tensorflow.python.framework.tensor_shape.TensorShape